Viết file @ShowOuput.js call tới API @routes.py @api.js 
để hiển thị các file trong thư mục @output 
----------------------

Viết file @ShowOuput.js call tới API @routes.py liên kết tới @app.py - và @api.js
để hiển thị các file trong thư mục @output 
----------------------

1. Giải thích vai trò của các file được nhắc đến bên trên: @routes.py liên kết tới @app.py - và @api.js

Let me explain the role of each file in this system:

1. **routes.py**:
- This is a Flask backend file that defines API endpoints for file operations
- Key functionalities:
  - `/upload` - Handles file uploads
  - `/list/<folder>` - Lists files in uploads/output folders
  - `/download/<folder>/<filename>` - Handles file downloads
- References in code:

```1:89:backend/api/files/routes.py
from flask import Blueprint, request, jsonify, send_file
import os
from werkzeug.utils import secure_filename
import pandas as pd

bp = Blueprint('files', __name__)

UPLOAD_FOLDER = os.path.join(os.path.dirname(__file__), '../../uploads')
OUTPUT_FOLDER = os.path.join(os.path.dirname(__file__), '../../output')
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

@bp.route('/upload', methods=['POST'])
def upload_file():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file uploaded'}), 400

        file = request.files['file']
        if not file or file.filename == '':
            return jsonify({'error': 'No file selected'}), 400

        if not file.filename.endswith(('.xlsx', '.xls')):
            return jsonify({'error': 'Invalid file type. Please upload Excel file'}), 400

        filename = secure_filename(file.filename)
        file_path = os.path.join(UPLOAD_FOLDER, filename)
        file.save(file_path)
        
        df = pd.read_excel(file_path)
        data = df.to_dict(orient='records')
        
        return jsonify({
            'success': True,
            'data': data, 
            'filename': filename
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@bp.route('/list/<folder>', methods=['GET'])
def list_files(folder):
    try:
        if folder not in ['uploads', 'output']:
            return jsonify({'error': 'Invalid folder'}), 400
            
        folder_path = UPLOAD_FOLDER if folder == 'uploads' else OUTPUT_FOLDER
        files = []
        
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                file_stats = os.stat(file_path)
                files.append({
                    'name': filename,
                    'size': file_stats.st_size,
                    'modified': file_stats.st_mtime
                })
                
        return jsonify({
            'success': True,
            'files': files
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@bp.route('/download/<folder>/<filename>', methods=['GET'])
def download_folder_file(folder, filename):
    try:
        if folder not in ['uploads', 'output']:
            return jsonify({'error': 'Invalid folder'}), 400
            
        folder_path = UPLOAD_FOLDER if folder == 'uploads' else OUTPUT_FOLDER
        file_path = os.path.join(folder_path, secure_filename(filename))
        
        if not os.path.exists(file_path):
            return jsonify({'error': 'File not found'}), 404
            
        return send_file(file_path, as_attachment=True)
        
    except Exception as e:
```


2. **app.py**:
- This is the main Flask application file
- Key responsibilities:
  - Initializes Flask app
  - Enables CORS (Cross-Origin Resource Sharing)
  - Registers blueprints (routes) with URL prefixes
  - Example: `files_bp` is registered with `/api/files` prefix
- References in code:

```1:14:backend/app.py
from flask import Flask
from flask_cors import CORS
from api.scripts import bp as scripts_bp
from api.files import bp as files_bp

app = Flask(__name__)
CORS(app)

# Register blueprints
app.register_blueprint(scripts_bp, url_prefix='/api/scripts')
app.register_blueprint(files_bp, url_prefix='/api/files')

if __name__ == '__main__':
    app.run(debug=True)
```


3. **api.js**:
- Frontend service file that communicates with backend APIs
- Key functions:
  - `getOutputFiles()` - Fetches list of files from output folder
  - `downloadFile()` - Initiates file download
- References in code:

```32:45:frontend/src/services/api.js
//  API cho Output

// Lấy danh sách files từ output folder
export const getOutputFiles = async () => {
    const response = await fetch(`${API_BASE_URL}/api/list/output`);
    const data = await response.json();
    if (!data.success) throw new Error(data.error);
    return data.files;
};

// Download file từ output folder
export const downloadFile = (filename) => {
    window.open(`${API_BASE_URL}/api/download/output/${filename}`, '_blank');
};
```


The connection flow works like this:

1. Frontend (`api.js`) makes HTTP requests to specific endpoints
2. `app.py` receives these requests and routes them based on URL prefixes
3. `routes.py` handles the actual logic for each endpoint
4. Data flows back to frontend through the same path

For example, when listing output files:
1. Frontend calls `getOutputFiles()` from `api.js`
2. Request goes to `/api/files/list/output`
3. `app.py` routes it to the files blueprint
4. `routes.py` handles the request and returns file list
5. Frontend receives and displays the data

This separation of concerns allows for:
- Clean code organization
- Easy maintenance
- Clear API structure
- Secure file operations
- Proper error handling at each level

Would you like me to proceed with writing the ShowOutput.js component that integrates with these files?

Let me explain what `scripts_bp` and `files_bp` are in the Flask application.

In Flask, `bp` stands for "Blueprint". Blueprints are a way to organize a Flask application into smaller, reusable components. Let's break down both blueprints:

1. **files_bp**:
- This is a blueprint for handling file-related operations
- Handles routes related to file uploads, downloads, and listing
- Registered with prefix `/api/files`
- Contains endpoints like:
  - `/api/files/upload`
  - `/api/files/list/output`
  - `/api/files/download/output/<filename>`

2. **scripts_bp**:
- This is a blueprint for handling script execution operations
- Handles routes related to running different content generation scripts
- Registered with prefix `/api/scripts`
- Contains endpoints like:
  - `/api/scripts/run/<scriptName>`

Here's how they're registered in app.py:
```python
app.register_blueprint(scripts_bp, url_prefix='/api/scripts')
app.register_blueprint(files_bp, url_prefix='/api/files')
```

The benefits of using blueprints:
1. Code Organization: Separates different functionalities into their own modules
2. URL Prefixing: All routes in a blueprint share a common URL prefix
3. Modularity: Each blueprint can have its own templates, static files, and error handlers
4. Reusability: Blueprints can be reused across different Flask applications

For example, when the frontend makes an API call:
- `getOutputFiles()` calls `/api/files/list/output` → handled by `files_bp`
- `runScriptApi()` calls `/api/scripts/run/<scriptName>` → handled by `scripts_bp`

This separation makes the code more maintainable and easier to understand.